In [ ]:
import re
import datetime
import random
import numpy as np
import pandas as pd
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing as prep
from datetime import date, timedelta , datetime
import math
import os 
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder 
import pickle
from datetime import datetime

In [ ]:
os.getcwd()

In [ ]:
computed_data_path = '../computed_data/'

if not os.path.exists(computed_data_path): 
    os.makedirs(computed_data_path)

In [ ]:
data_original  = pd.read_csv('../data/donors.csv')
data = data_original.copy()


In [ ]:
data.CONTROLN

In [ ]:

data.set_index('CONTROLN', inplace=True)

In [ ]:
data

In [ ]:
#description = ['ODATEDW' , 'OSOURCE' , 'TCODE' , 'STATE' , 'ZIP' , 'MAILCODE' , 'PVASTATE' , 'DOB' , 'NOEXCH' , 'RECINHSE' , 'RECP3' , 'RECPGVG' , 'RECSWEEP','MDMAUD' , 'DOMAIN' , 'HOMEOWNR' , 'CHILD03' , 'CHILD07' , 'CHILD12' , 'CHILD18' , 'NUMCHLD' , 'INCOME' , 'GENDER' , 'WEALTH1' , 'HIT' ,'MBCRAFT' , 'MBGARDEN' , 'MBBOOKS' , 'MBCOLECT' , 'MAGFAML' , 'MAGFEM' , 'MAGMALE' , 'PUBGARDN' , 'PUBCULIN' , 'PUBHLTH' , 'PUBDOITY' ,'PUBNEWFN' , 'PUBPHOTO', 'PUBOPP' ,'DATASRCE' , 'MALEMILI' , 'MALEVET' , 'VIETVETS' , 'WWIIVETS' , 'LOCALGOV' , 'STATEGOV' , 'FEDGOV' , 'SOLP3' ,'SOLIH' , 'MAJOR' , 'WEALTH2' , 'GEOCODE' , 'COLLECT1' , 'VETERANS' , 'BIBLE', 'CATLG' , 'HOMEE' , 'PETS', 'CDPLAY' , 'STEREO' , 'PCOWNERS' , 'PHOTO' , 'CRAFTS' , 'FISHER' , 'GARDENIN' , 'BOATS' , 'WALKER' , 'KIDSTUFF' , 'CARDS' , 'PLATES' , 'LIFESRC' , 'PEPSTRFL']
numeric_features = ['NUMCHLD','INCOME' , 'WEALTH1' , 'MBCRAFT' , 'MBGARDEN' , 'MBBOOKS' , 'MBCOLECT' ,
                    'MAGFAML' , 'MAGFEM' , 'MAGMALE' , 'PUBGARDN' , 'PUBCULIN' , 'PUBHLTH' , 'PUBDOITY' , 'PUBNEWFN' , 'PUBOPP' , 'WEALTH2' ]
boolean_features = [ 'PEPSTRFL', 'MAILCODE', 'PVASTATE', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP', 'MAJOR']
donors_interests = ['COLLECT1','VETERANS' , 'BIBLE' , 'CATLG' , 'HOMEE' , 'PETS' , 'CDPLAY','STEREO' , 'PCOWNERS' , 'PHOTO' , 'CRAFTS' , 'FISHER' , 'GARDENIN' , 'BOATS' , 'WALKER' , 'KIDSTUFF' , 'CARDS'  , 'PLATES']

In [ ]:
colors = [ '#4C72B0' , '#dd8452' , '#55A868' , '#C44E52']

In [ ]:
#Number of missing values in each feature
sns.set()

#figure and axis
fig, (ax1) = plt.subplots(1, figsize=(20,6))

#data
ax1_data=data[numeric_features].isna().sum().sort_values(ascending=False)
ax1.bar(x=ax1_data.index, height=ax1_data)
ax1.set_title("Missing Values per Variable", fontsize=20)
ax1.set_ylabel("Total Missing Values")
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)

for tick in ax1.get_xticklabels():
    tick.set_rotation(-55)
    
plt.show()

# CHECK INCOHERENCES

DONATION DATES BEFORE DONOR WAS BORN

In [ ]:
data.loc[( pd.to_datetime(data.DOB) ) > ( pd.to_datetime(data.ODATEDW) )][['DOB','ODATEDW']]

DONOR WITH LESS THAN 14 (MINIMUM AGE TO WORK IN THE US) 

In [ ]:
data.loc[ (pd.to_datetime(data.DOB).dt.year > 2006 )]

DONORS WITH LESS THAN 14 WHICH ARE HOMEOWNERS

In [ ]:
data.loc[ (pd.to_datetime(data.DOB).dt.year > 2006 ) & (data.HOMEOWNR == 'H' ) ][['DOB','HOMEOWNR']]

DONORS WITH LESS THAN 14 WHICH RECEIVE AN INCOME (14 IS THE MINIMUM AGE TO WORK IN THE US )

In [ ]:
data.loc[ (pd.to_datetime(data.DOB).dt.year > 2006 ) & (data.INCOME > 1 ) ][['DOB','INCOME']]

CHECK FOR SITUATIONS WHERE THE NUMBER OF CHILDRENS DECLARED ON COLUMNS 'CHILD03' , 'CHILD07' , 'CHILD12' AND 'CHILD18' IS BIGGER THAN THE CHILDREN DECLARED ON NUMCHLD

In [ ]:
def checkChildrenInconsistencies(x):  
    numberOfChildren = 0
    colsToCheck = ['CHILD03' , 'CHILD07' , 'CHILD12' , 'CHILD18' ]
    for i in colsToCheck:
        if (x[i] == 'B'):
            numberOfChildren = numberOfChildren + 2
        elif (x[i] == 'M' or x[i] == 'F'):
            numberOfChildren = numberOfChildren + 1
            
    return numberOfChildren
  
    
data['MINIMUM_POSSIBLE_CHILD'] = data.apply(lambda x: checkChildrenInconsistencies(x) , axis=1)
data.loc[ data.MINIMUM_POSSIBLE_CHILD > data.NUMCHLD][['CHILD03' , 'CHILD07' , 'CHILD12' , 'CHILD18', 'NUMCHLD' , 'MINIMUM_POSSIBLE_CHILD'  ]]

In [ ]:
del data['MINIMUM_POSSIBLE_CHILD']
numeric_features.remove('NUMCHLD')

In [ ]:
# NOEXCH values not documented , only possible values are " " and "X"
data.NOEXCH = data.NOEXCH.astype(str)

sns.set()
sns.countplot(x="NOEXCH", data=data)
plt.suptitle("NOEXCH Frequencies")
plt.show()

# DATA PREPARATION

#### CARDINALITY REDUCTION 

In [ ]:
# GENDER
plt.title("Gender Distribution")
sns.countplot(x = 'GENDER',
              data = data , order = data.GENDER.value_counts().index )
plt.show()


In [ ]:
data.GENDER = data.GENDER.apply(lambda x: 'M' if x == 'M' else 'F' if x == 'F' else 'O' )

#### FEATURE ENGINEERING

AS COLUMN DOMAIN HAS SPECIFIC VALUE FOR EACH CHARACTER, WE WILL SPLIT IN TWO COLUMNS AND RESCALE THE SECOND INTO A UNIFORM SCALE WITHOUT THE PECULIARITIES FOR THE URBAN COMMUNITIES

In [ ]:
#DOMAIN
data['DOMAIN_1'] = data.DOMAIN.astype(str).str[0]
data['DOMAIN_2'] = data.DOMAIN.astype(str).str[1]

data['DOMAIN_2'] = ['5' if b == '4' or (b == '3' and a != 'U' ) else b for a,b in zip(data['DOMAIN_1'] , data['DOMAIN_2'] ) ]
data['DOMAIN_2'] = ['4' if a == 'U' and b == '3' else b for a, b in zip(data['DOMAIN_1'] , data['DOMAIN_2'] ) ]
data['DOMAIN_2'] = ['3' if b == '2' and a != 'U' else b for a, b in zip(data['DOMAIN_1'] , data['DOMAIN_2'] ) ]

data['DOMAIN_2'] = pd.to_numeric(data['DOMAIN_2'], errors='coerce')
numeric_features.append('DOMAIN_2')

CREATE COLUMN AGE BASED ON 'DOB'

In [ ]:
#We assumed the current date as 2020/12/31
import datetime

x = datetime.date(2020, 12, 31)

def findAge(dob ):
    if str(dob) == "NaT" :
        return np.nan
    else :
        age = (x - dob.date() ) // timedelta(days=365.2425)
        return age    
    
    

data['DOB'] = pd.to_datetime(data.DOB)
data['AGE'] = data.apply(lambda x: findAge(x.DOB ), axis=1)
numeric_features.append('AGE')

CREATE COLUMN YEARS_SINCE_FIRST_DONATION BASED ON 'DATEDW'

In [ ]:
from datetime import datetime

def yearsSinceFirstDonor(odatedw):
    date_transform = datetime.strptime(odatedw, '%Y-%m-%d').date()
    years = (x - date_transform ) // timedelta(days=365.2425)
    return years   
    
    
data['YEARS_SINCE_FIRST_DONATION'] = data.apply(lambda x: yearsSinceFirstDonor(x.ODATEDW), axis=1)
numeric_features.append('YEARS_SINCE_FIRST_DONATION')

In [ ]:
plt.title("YEARS_SINCE_FIRST_DONATION Distribution")
sns.countplot(x = 'YEARS_SINCE_FIRST_DONATION',
              data = data , order = data.YEARS_SINCE_FIRST_DONATION.value_counts().index )
plt.show()

#### BINARY TRANSFORMATION

In [ ]:
for col_name in donors_interests:
    data[col_name] = data[col_name].apply(lambda x: 1 if x == 'Y' else 0)
    data[col_name] = data[col_name].astype(str).astype(int)
    
data.PVASTATE = data.PVASTATE.apply(lambda x: 1 if x == 'P' else 1 if x == 'E' else 0 )

x_or_u = ['RECINHSE', 'RECP3' , 'RECPGVG' , 'RECSWEEP' , 'MAJOR' , 'PEPSTRFL' ]
for i in x_or_u:
    data[i] = data[i].apply(lambda x: 1 if x == 'X' else 0 )
    
data.MAILCODE = data.MAILCODE.apply(lambda x: 1 if x == 'B' else 0 )
data.PVASTATE = data.PVASTATE.apply(lambda x: 1 if x == 'P' else 1 if x == 'E' else 0 )

#### CHANGING EMPTY CLASSES (' ') VALUES 

In [ ]:
data.LIFESRC = data.LIFESRC.apply(lambda x: '1' if x == '1' else '2' if x == '2' else '3' if x == '3' else '0' )
data.DATASRCE = data.DATASRCE.apply(lambda x: '1' if x == '1' else '2' if x == '2' else '3' if x == '3' else '0' )
data.SOLIH = data.SOLIH.apply(lambda x: '13' if x == ' ' else x )
data.SOLIH = data.SOLIH.astype(str).astype(int)
numeric_features.append('SOLIH')
data.SOLP3 = data.SOLP3.apply(lambda x: '13' if x == ' ' else x )
data.SOLP3 = data.SOLP3.astype(str).astype(int)
numeric_features.append('SOLP3')
data.HOMEOWNR = data.HOMEOWNR.apply(lambda x: 'H' if x == 'H' else 'U' if x == 'U' else 'O' )

#### ONE HOT ENCODE 

In [ ]:
data_ohc = data.copy()
to_encode = [ 'GENDER'  , 'DOMAIN_1' , 'DATASRCE' , 'LIFESRC' , 'HOMEOWNR']

ohc = OneHotEncoder(sparse=False, drop="first")
ohc_feat = ohc.fit_transform(data_ohc[to_encode])
ohc_feat_names = ohc.get_feature_names()
ohc_df = pd.DataFrame(ohc_feat, index=data_ohc.index, columns=ohc_feat_names) 
ohc_df

#GENDER ->  x0_M ; x0_O
#DOMAIN_1 -> x1_C ; x1_R ; x1_S ;  x1_T ; x1_U
#DATASRCE -> x2_1 ; x2_2 ; x2_3
#LIFESRC -> x3_1 ; x3_2 ; x3_3
#HOMEOWNR -> x4_O ; x4_U

data_ohc = pd.concat([data_ohc.drop(columns=to_encode), ohc_df], axis=1)
data = data_ohc

data = data.rename(columns={'x0_M':'GENDER_M', 'x0_O':'GENDER_O', 'x1_C':'DOMAIN_1_C', 'x1_R':'DOMAIN_1_R', 'x1_S':'DOMAIN_1_S' , 'x1_T':'DOMAIN_1_T', 'x1_U':'DOMAIN_1_U', 'x2_1':'DATASRCE_1',
                  'x2_2':'DATASRCE_2', 'x2_3':'DATASRCE_3', 'x3_1':'LIFESRC_1', 'x3_2':'LIFESRC_2', 'x3_3':'LIFESRC_3' , 'x4_O':'HOMEOWNR_O', 'x4_U':'HOMEOWNR_U'})


new_col_names = ['GENDER_M' , 'GENDER_O' , 'DOMAIN_1_C' , 'DOMAIN_1_R' , 'DOMAIN_1_S', 'DOMAIN_1_T' , 'DOMAIN_1_U' , 'DATASRCE_1', 'DATASRCE_2' , 'DATASRCE_3' , 'LIFESRC_1', 'LIFESRC_2','LIFESRC_3' , 'HOMEOWNR_O' , 'HOMEOWNR_U']

boolean_features = [ 'PEPSTRFL', 'MAILCODE', 'PVASTATE', 'RECINHSE', 'RECP3', 'RECPGVG', 'RECSWEEP', 'MAJOR']


description = numeric_features + new_col_names + boolean_features + donors_interests

In [ ]:
data['STATE']

In [ ]:
with open(os.path.join(computed_data_path, 'description_features.pickle'), 'wb') as f:
       pickle.dump(data[description], f)

data = pd.merge(data[['STATE' , 'ZIP']] , data[description] , left_index=True, right_index=True)

In [ ]:
def cluster_profiles(df, label_columns, figsize, compar_titles=None):
    """
    Pass df with labels columns of one or multiple clustering labels. 
    Then specify this label columns to perform the cluster profile according to them.
    """
    if compar_titles == None:
        compar_titles = [""]*len(label_columns)
        
    sns.set()
    fig, axes = plt.subplots(nrows=len(label_columns), ncols=2, figsize=figsize, squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compar_titles):
        # Filtering df
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df.drop(drop_cols, axis=1)
        
        # Getting the cluster centroids and counts
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]
        
        # Setting Data
        pd.plotting.parallel_coordinates(centroids, label, color=sns.color_palette(), ax=ax[0])
        sns.barplot(x=label, y="counts", data=counts, ax=ax[1])

        #Setting Layout
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(len(handles))]
        #print('cluster_labels: ', cluster_labels)
        
        color_mapper = {}
        for handle in handles: 
            color_mapper[handle.get_label()] = handle.get_color()
        
        #print('handles: ', handles[0].get_color())
        #print('handles: ', handles[0].get_label())


        ax[0].annotate(text=titl, xy=(0.95,1.1), xycoords='axes fraction', fontsize=13, fontweight = 'heavy') 
        ax[0].legend(handles, cluster_labels) # Adaptable to number of clusters
        ax[0].axhline(color="black", linestyle="--")
        ax[0].set_title("Cluster Means - {} Clusters".format(len(handles)), fontsize=13)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=30)
        ax[1].set_xticklabels(cluster_labels)
        ax[1].set_xlabel("")
        ax[1].set_ylabel("Absolute Frequency")
        ax[1].set_title("Cluster Sizes - {} Clusters".format(len(handles)), fontsize=13)
    
    #plt.subplots_adjust(hspace=0.4, top=0.90)
    plt.tight_layout()
    plt.title = ''
#    plt.suptitle("Cluster Simple Profilling", fontsize=23)

    filename = ''.join(label_columns)
    plt.savefig(os.path.join(f'profile_mean_{filename}.jpeg'), dpi=200)

    plt.show()
    
    return color_mapper

In [ ]:
data[donors_interests].head()

In [ ]:
clusters  = pd.read_csv('computed_data/cluster_donor_mapping.csv' )
clusters.set_index('CONTROLN', inplace=True)
clusters

In [ ]:
clusters_merged = pd.merge(data, clusters, left_index=True, right_index=True)

In [ ]:
data.index.intersection(clusters.index)

In [ ]:
data

In [ ]:
clusters

In [ ]:
clusters_merged

In [ ]:
scaler = MinMaxScaler()
scaler.fit(clusters_merged[donors_interests])
scaled_data = scaler.transform(clusters_merged[donors_interests])

clusters_merged[donors_interests] = scaled_data

# Profilling each cluster (product, behavior, merged)
cluster_profiles(
    df = clusters_merged[ donors_interests + ['merged_labels']], 
    label_columns = ['merged_labels'], 
    figsize = (28, 13), 
    compar_titles = ["Donor interests clustering"]
)

In [ ]:
personal_details = ['GENDER_M', 'GENDER_O', 'DOMAIN_1_C' , 'DOMAIN_1_R' , 'DOMAIN_1_S', 'DOMAIN_1_T' , 'DOMAIN_1_U', 'DOMAIN_2' , 'HOMEOWNR_O' , 'HOMEOWNR_U' , 'INCOME' , 'AGE' , 'YEARS_SINCE_FIRST_DONATION']

secondScaler = MinMaxScaler()
secondScaler.fit(clusters_merged[personal_details])
scaled_data_2 = secondScaler.transform(clusters_merged[personal_details])

clusters_merged[personal_details] = scaled_data_2

# Profilling each cluster (product, behavior, merged)
cluster_profiles(
    df = clusters_merged[personal_details + ['merged_labels']], 
    label_columns = ['merged_labels'], 
    figsize = (28, 13), 
    compar_titles = ["Personal Details clustering"]
)

In [ ]:
clusters_merged[['STATE','merged_labels','neigh_labels', 'hist_labels']]

In [ ]:
geo = pd.DataFrame(clusters_merged.groupby(['STATE','merged_labels']).size().to_frame('SIZE') )
geo = geo.reset_index()

states  = pd.read_csv('data/usa_states.csv' , sep=';')

states.head()
geographic_info = pd.merge(geo, states, left_on='STATE', right_on='state')
geographic_info = geographic_info.drop(columns=['state'])
geographic_info

In [ ]:
import plotly.graph_objects as go


limits = geographic_info.merged_labels.unique().tolist()

lat = [0 , 0 , 0.7 , -0.7]
lon = [0.5 , -0.5 , 0 , 0]
cities = []
scale = 4

fig = go.Figure()

for i in limits:    
    lim = limits[i]
    df_sub = geographic_info.loc[(geographic_info.merged_labels == lim)]
    fig.add_trace(go.Scattergeo(
        locationmode = 'USA-states',
        lon = df_sub['longitude'] + lon[i],
        lat = df_sub['latitude'] + lat[i],
        text = df_sub[[ "name", 'SIZE']],
        marker = dict(
            size = df_sub['SIZE']/scale,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = 'Cluster - {0} '.format(lim)))
        
fig.update_layout(
        title_text = 'Cluster Distribution per State',
        showlegend = True,
        geo = dict(
            scope = 'usa',
            landcolor = 'rgb(217, 217, 217)',
        )
    )

fig.show()

In [ ]:
a = geographic_info.groupby(['STATE'])['SIZE'].transform(max) == geographic_info['SIZE']
biggestByState = geographic_info[a]
biggestByState

In [ ]:
import plotly.graph_objects as go

import pandas as pd


fig = go.Figure(data=go.Choropleth(
    locations=biggestByState['STATE'], # Spatial coordinates
    z = biggestByState['merged_labels'].astype(float), # Data to be color-coded
    text = biggestByState[[ "name", 'SIZE']],
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = colors    
))

fig.update_layout(
    title_text = 'Cluster Dominance by State',
    geo_scope='usa', # limite map scope to USA
            showlegend = False

    
)

fig.show()

In [ ]:
biggestByState.loc[ biggestByState.STATE == 'AZ' ]

In [ ]:
check = clusters_merged[['STATE','ZIP','merged_labels']]

In [ ]:
geo_zip = pd.DataFrame(check.groupby(['ZIP','merged_labels']).size().to_frame('SIZE') )
geo_zip = geo_zip.reset_index()
geo_zip.ZIP = geo_zip.ZIP.apply(lambda x : x[ 0 : 5])
geo_zip.ZIP = geo_zip.ZIP.astype(str).astype(int)
fip  = pd.read_csv('data/zip_fip.csv')
fip = fip[['ZIP','STCOUNTYFP','STATE']]
fip = fip.rename(columns={"STATE": "STATE_ZIP", "STCOUNTYFP": "FIP"})
geographic_fip = pd.merge(geo_zip, fip, left_on='ZIP', right_on='ZIP')
geographic_fip

a = geographic_fip.groupby(['ZIP'])['SIZE'].transform(max) == geographic_fip['SIZE']
biggestByFIP = geographic_fip[a]
biggestByFIP

In [ ]:
if False: 

    import plotly.figure_factory as ff

    import numpy as np
    import pandas as pd


    fips = biggestByFIP['FIP'].tolist()
    values = biggestByFIP['merged_labels'].tolist()


    fig = ff.create_choropleth(
        fips=fips, values=values, scope=['usa'],
        colorscale=colors,
        show_state_data=False,
        show_hover=True,
        asp = 2.9,
        title_text = 'Clustering by ZIP Code',
        legend_title = 'Clusters'
    )

    fig.layout.template = None
    fig.show()

In [ ]:
if False: 

    import plotly.figure_factory as ff

    fips = biggestByFIP.FIP.tolist()
    values = biggestByFIP.merged_labels.tolist()

    fig = ff.create_choropleth(fips=fips, values=values,
        colorscale=colors ,
        show_state_data=True,
        show_hover=True, title_text = 'Clustering by ZIP CODE', legend_title = 'Clusters')
    fig.layout.template = None
    fig.show()